In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [3]:
import os

os.chdir('/usr/local/lib/python3.10/dist-packages/konlpy/java')
os.getcwd()

'/usr/local/lib/python3.10/dist-packages/konlpy/java'

In [4]:
# 임시 폴더에 Okt 사전 파일 압축 풀기
!pwd

/usr/local/lib/python3.10/dist-packages/konlpy/java


In [5]:
import shutil

filename = 'open-korean-text-2.1.0.jar'
src = '/content/drive/MyDrive/data/'
dir = '/usr/local/lib/python3.10/dist-packages/konlpy/java/'
shutil.copy(src + filename, dir + filename)

'/usr/local/lib/python3.10/dist-packages/konlpy/java/open-korean-text-2.1.0.jar'

In [6]:
# 테스트
from konlpy.tag import Okt
okt = Okt()
print(okt.nouns('문딸랑 좌좀비 문베 기집년닭근혜 레즈게이 더듬어강간당박사모'))
print(okt.morphs('문딸랑 좌좀비 문베 기집년닭근혜 레즈게이 더듬어강간당박사모'))

['문딸랑', '좌좀비', '문베', '기집년', '닭근혜', '레즈', '게이', '더듬어강간당', '박사모']
['문딸랑', '좌좀비', '문베', '기집년', '닭근혜', '레즈', '게이', '더듬어강간당', '박사모']


In [7]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib as mpl

path = '/content/drive/MyDrive/data/programmers_ai/yujatea/comp_2/'

In [8]:
train = pd.read_csv(os.path.join(path,'train.csv'))

In [9]:
vocab = pd.read_csv(os.path.join(path, 'vocab.csv'))

In [10]:
vocab

,key,replace
0,SiBal,시발
1,ssibark,시발
2,개sae들,개새들
3,Hannam,한남
4,sochu,소추
...,...,...
439,쪽 빠리,쪽바리
440,쉐 퀴,새끼
441,양아취새끼,양아치새끼
442,병 ㅅ들,병신들


In [11]:
vocab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 444 entries, 0 to 443
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   key      444 non-null    object
 1   replace  444 non-null    object
dtypes: object(2)
memory usage: 7.1+ KB


In [12]:
def text_preprocess(df, vocab):
  # 사용자 정의 지정 표현 수동으로 변환시켜주기
  for key, replace in zip(vocab['key'].values, vocab['replace'].values):
    try:
      df['text'] = df['text'].str.replace(key, replace, regex=True)
    except:
      print(key)

  # ') 깨문', '병@(#%^신' 처리
  df['text'] = df['text'].replace({r'\) 깨문': '대깨문'}, regex=True)
  df['text'] = df['text'].str.replace(r'병[@#%^]?\(*\)?.*?신', "병신", regex=True)

  # "OO"를 제외한 영어 표현은 EDA 결과 대체로 label 6에 해당 따라서 일괄적으로 토큰화 대상에서 제외 시켜준다.
  # 사실은 어떻게 핸들링을 해야할지 구체적인 방법을 모르겠다.
  # ?와 !는 어떤 의미를 가질것이라 추측하고 남긴다.
  # 나머지 특수기호들은 전부 지운다.
  df['text'] = df['text'].replace('[^0-9가-힣!?\sOO]+', '', regex=True)

  # '?'와 '!'가 연속해서 나오는 경우 1개만 남기고 지워주기
  df['text'] = df['text'].str.replace(r'!+', '!', regex=True)
  df['text'] = df['text'].str.replace(r'\?+', '?', regex=True)

  # 'ㅋㅋㅋ'나 'ㅎㅎㅎ' 처럼 한글 자음으로만 이루어진 경우는 지워주기
  df['text'] = df['text'].str.replace('[ㄱ-ㅎ]+', '?', regex=True)

  # 모음으로만 이루어진 부분들 지워주기
  df['text'] = df['text'].str.replace('[ㅏ-ㅣ]+', '?', regex=True)

In [13]:
# # 사용자 정의 지정 표현 수동으로 변환시켜주기
# for key, replace in zip(vocab['key'].values, vocab['replace'].values):
#   try:
#     train['text'] = train['text'].str.replace(key, replace, regex=True)
#   except:
#     print(key)

In [14]:
# ') 깨문', '병@(#%^신' 처리, 'X', 'x'
# train['text'] = train['text'].replace({r'\) 깨문': '대깨문'}, regex=True)
# train['text'] = train['text'].str.replace(r'병[@#%^]?\(*\)?.*?신', "병신", regex=True)

In [15]:
# 영어 표현은 EDA 결과 대체로 label 6에 해당 따라서 일괄적으로 토큰화 대상에서 제외 시켜준다
# 사실은 어떻게 핸들링을 해야할지 구체적인 방법을 모르겠다.
# ?와 !는 어떤 의미를 가질것이라 추측하고 남긴다.
# 나머지 특수기호들은 전부 지운다.
# train['text'] = train['text'].replace('[^0-9가-힣!?\s]+', '', regex=True)

In [16]:
# '?'와 '!'가 연속해서 나오는 경우 1개만 남기고 지워주기
# train['text'] = train['text'].str.replace(r'!+', '!', regex=True)

In [17]:
# train['text'] = train['text'].str.replace(r'\?+', '?', regex=True)

In [18]:
# 'ㅋㅋㅋ'나 'ㅎㅎㅎ' 처럼 한글 자음으로만 이루어진 경우는 지워주기
# train['text'] = train['text'].str.replace('[ㄱ-ㅎ]+', '?', regex=True)

In [19]:
# 모음으로만 이루어진 부분들 지워주기
# train['text'] = train['text'].str.replace('[ㅏ-ㅣ]+', '?', regex=True)

In [20]:
text_preprocess(train, vocab)

) 깨문
병@(#%^신


In [21]:
test = pd.read_csv(os.path.join(path, 'test.csv'))
test.head()

,ID,text,label
0,0,"""솔직히 우리나라 청년들도 불쌍하고 아재들도 불쌍하고 노인들도 불쌍하다. 나라가 참""",NaN
1,1,그만 보고싶네요 .늙은애들은.,NaN
2,2,"""더러운 개신교벌레 새퀴""",NaN
3,3,근데전태수씨 사망이유가뭔가요그어떤기사에도 나오질않네요,NaN
4,4,"""태극기부대와 틀닭바퀴충들에게 순시리는 국모다. ㅉㅉ""",NaN


In [22]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13491 entries, 0 to 13490
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      13491 non-null  int64  
 1   text    13491 non-null  object 
 2   label   0 non-null      float64
dtypes: float64(1), int64(1), object(1)
memory usage: 316.3+ KB


In [23]:
# 특수문자로만 이루어진 케이스
pattern = r'^[\W_]+$'
matches = test[test['text'].str.contains(pattern, regex=True)]
matches

,ID,text,label
1415,1415,"""👏👏👏👏👏👏👏🇰🇷🇰🇷🇰🇷🇰🇷🇰🇷🇰🇷🇰🇷""",NaN
12955,12955,"""🤣🤣🤣🤣🤣🤣🤣😋😋😋😋😋""",NaN
13402,13402,"""👏🏻👏🏻👏🏻👏🏻👏🏻""",NaN


In [24]:
matches_eng = test[test['text'].str.contains('[a-zA-Z]', regex=True)]
matches_eng

,ID,text,label
48,48,"""이희호같은 OOO 페미년도 거의 백살까지 살았는데 엔두는 120까지 가야지 ㄹㅇ ...",NaN
141,141,나도 요즘 나이가 좀 찬 30대라 막 챙겨보는사람은 아니고 처음에는BJ들 안좋게 봤...,NaN
143,143,"""Chinese cao ni ma !""",NaN
155,155,"""low 병 x 대댓글 자삭했네 ㅉㅉ 대깨문들은 걍 죽어줬으면 좋겠다""",NaN
231,231,"""전형적인 쪽발이 새xxx끼 관상이다. 하는짓도 생긴것과 똑같네""",NaN
...,...,...,...
13389,13389,"""OOO들 우리할머니 발뒤꿈치 보다 더 못생긴년들""",NaN
13425,13425,법을강화안시키니 이모양이꼴이지 뭐하는거니 도대체OOO,NaN
13454,13454,"""그뇬 이라고 한 걸레도 있었는데 저OOO 해도 되지모""",NaN
13482,13482,"""50대 OOO 들 왜 사냐 도대체 니들은""",NaN


In [25]:
text_preprocess(test, vocab)

) 깨문
병@(#%^신


In [26]:
test

,ID,text,label
0,0,솔직히 우리나라 청년들도 불쌍하고 아재들도 불쌍하고 노인들도 불쌍하다 나라가 참,NaN
1,1,그만 보고싶네요 늙은애들은,NaN
2,2,더러운 개신교벌레 새끼,NaN
3,3,근데전태수씨 사망이유가뭔가요그어떤기사에도 나오질않네요,NaN
4,4,태극기부대와 틀닭바퀴충들에게 순시리는 국모다,NaN
...,...,...,...
13486,13486,시발저년 한국인 아니라서 법적으로 어떻게 못하는거아님?OO아 시발 느그나라로 꺼져,NaN
13487,13487,틀딱 택시기사 왔는가,NaN
13488,13488,틀딱들의 흔한 망상중 하나죠,NaN
13489,13489,빨갱이 새끼새끼 역시 잔인하노 능지처참은 중국놈인 니놈이고,NaN


In [27]:
from sklearn.model_selection import train_test_split

train_df, valid_df = train_test_split(train, test_size=0.2, random_state=42, stratify=train['label'])

In [28]:
train_df.shape, valid_df.shape

((52690, 3), (13173, 3))

In [29]:
# 토크나이저 정의
from konlpy.tag import Okt
okt = Okt()

In [30]:
import torch
import torchtext
from torchtext import data
import torch.nn as nn
import torch.nn.functional as F

import random

In [31]:
seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

In [32]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [33]:
tokenizer = get_tokenizer(okt.morphs)
tokens = tokenizer("OOO들 우리할머니 발뒤꿈치 보다 더 못생긴년들"	)
tokens

['OOO', '들', '우리', '할머니', '발뒤꿈치', '보다', '더', '못', '생긴', '년', '들']

In [47]:
train_iter = train['text'].values

def yield_tokens(data_iter):
    for text in data_iter:
      yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), min_freq=2,  specials=["<모름>"])

In [49]:
vocab.set_default_index(vocab["<모름>"])

In [50]:
text_pipeline = lambda x: vocab(tokenizer(x))

In [54]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [58]:
len(vocab)

30578

In [60]:
text_pipeline("OOO들 우리할머니 발뒤꿈치 보다 더 못생긴년들")

[13714, 1, 158, 1105, 0, 95, 54, 13, 146, 35, 1]

In [62]:
class CustomDataset(Dataset):
    def __init__(self, df, is_test=False):
        super().__init__()
        self.df = df
        self.is_test = is_test
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        text = text_pipeline(self.df.iloc[idx, 1])

        if self.is_test:
            return text
        else:
            label = self.df.iloc[idx, 2]
            return text, label

In [64]:
dataset_train = CustomDataset(train_df, is_test=False)
dataset_valid = CustomDataset(valid_df, is_test=False)

In [65]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

In [66]:
batch_size = 64

loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, worker_init_fn=seed_worker, generator=g, num_workers=2)
loader_valid = DataLoader(dataset_valid, batch_size=batch_size, shuffle=False, worker_init_fn=seed_worker, generator=g, num_workers=2)

In [ ]:
INPUT_DIM = len(vocab) #30578
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1